In [59]:
import os
import numpy as np
import cv2 as cv
import random

import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from torch import cuda, device
from torch import nn
from torch import optim, from_numpy, tensor

batch_size = 64

In [117]:
def read_training_data(data_directory, split, classes, img_size, model_name):
    '''
    Read training images and classes into multi-dimensional array.
    Images are compressed to img_size x img_size.
    '''
    split_data = [[], [], []]
    classes_data = [[], [], []]

    for spl_index, spl in enumerate(split):
        spl_path = os.path.join(data_directory, spl)
        for cla_index, cla in enumerate(classes):
            path = os.path.join(data_directory, spl, cla)
            if classes.index(cla) == 0:
                class_num = np.array([[1.], [0.]])
            else:
                class_num = np.array([[0.], [1.]])
            # class_num = classes.index(cla) # !!! replaced by if else statement
            for img in os.listdir(path):
                img_array = cv.imread(os.path.join(
                    path, img), cv.IMREAD_GRAYSCALE)
                img_array = cv.resize(img_array, (img_size, img_size))
                if model_name == 'base':
                    img_array = np.reshape(
                        img_array, (img_array.shape[0], img_array.shape[1], 1))
                elif model_name == 'torch':
                    img_array = np.reshape(
                        img_array, (1, img_array.shape[0], img_array.shape[1]))
                img_array = img_array.astype("float32") / 255
                split_data[spl_index].append(img_array)
                classes_data[spl_index].append(class_num)
        print(spl_path, '(read', len(classes_data[spl_index]), 'images)')

    for i in range(0, len(split_data)):
        comb_list = list(zip(split_data[i], classes_data[i]))
        random.shuffle(comb_list)
        split_data[i], classes_data[i] = zip(*comb_list)

    return split_data, classes_data


def reshape_img(img):
    '''
    Reshape image to 3D array with third dimension as 1.
    '''
    return np.reshape(img, (img.shape[0], img.shape[1], 1))

In [118]:
data_directory = '../data/'
split = ['train', 'val', 'test']
classes = ['NORMAL', 'PNEUMONIA']

In [119]:
split_data, classes_data = read_training_data(data_directory, split, classes, 100, 'torch')

../data/train (read 3756 images)
../data/val (read 417 images)
../data/test (read 1043 images)


In [120]:
x_train, y_train = split_data[0], classes_data[0]
x_val, y_val = split_data[1], classes_data[1]
x_test, y_test = split_data[2], classes_data[2]

In [128]:
class TorchCNN(nn.Module):
    '''
    '''
    
    def __init__(self):
        super(TorchCNN, self).__init__()
        self.c1 = nn.Conv2d(1, 32, 3)
        self.sig1 = nn.Sigmoid()
        self.pool1 = nn.MaxPool2d(2)
        self.c2 = nn.Conv2d(32, 64, 3)
        self.sig2 = nn.Sigmoid()
        self.pool2 = nn.MaxPool2d(2)
        self.f1 = nn.Linear(64 * 23 * 23, 128)
        self.sig3 = nn.Sigmoid()
        self.f2 = nn.Linear(128, 2)

    def forward(self, fw):
        '''
        '''
        fw = self.c1(fw)
        fw = self.sig1(fw)
        fw = self.pool1(fw)
        fw = self.c2(fw)
        fw = self.sig2(fw)
        fw = self.pool2(fw)
        fw = fw.view(fw.size(0), -1)
        fw = self.f1(fw)
        fw = self.sig3(fw)
        fw = self.f2(fw)
        return fw
    
def torch_cnn_prepare_data(split_data, classes_data, batch_size):
    '''
    '''
    x_train, y_train = split_data[0], classes_data[0]
    x_val, y_val = split_data[1], classes_data[1]
    x_test, y_test = split_data[2], classes_data[2]

    train_data = TensorDataset(tensor(x_train), tensor(y_train))
    train_load = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_data = TensorDataset(tensor(x_val), tensor(y_val))
    val_load = DataLoader(val_data, batch_size=batch_size, shuffle=True)
    test_data = TensorDataset(tensor(x_test), tensor(y_test))
    test_load = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    print('... created DataLoader for train, val and test.')

    return train_load, val_load, test_load

# def run_torch_cnn(split_data, classes_data, epochs, learning_rate, batch_size):
    '''
    '''

learning_rate = 0.01
epochs = 5
batch_size = 32

train_load, val_load, test_load = torch_cnn_prepare_data(split_data, classes_data, batch_size)
base_torch_model = TorchCNN()

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_torch_model.parameters(), lr=learning_rate)

device = torch.device("cuda" if cuda.is_available() else "cpu")
base_torch_model.to(device)

#

for epoch in range(epochs):
    base_torch_model.train()
    loss_run = 0.0
    for images, labels in train_load:
        images, labels = images.to(device), labels.squeeze().to(device)
        optimizer.zero_grad()
        out = base_torch_model(images)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        loss_run += loss.item()
    loss_res = loss_run / len(train_load)
    print('train finished...')
    base_torch_model.eval()
    true, all = 0, 0
    with torch.no_grad():
        for images, labels in val_load:
            images, labels = images.to(device), labels.squeeze().to(device)
            out = base_torch_model(images)
            _, predicted = torch.max(out.data, 1)
            all += labels.size(0)
            true += (predicted == labels.squeeze()).sum().item()
    acc = true / all
    print('acc', acc)

... created DataLoader for train, val and test.
train finished...
out tensor([[-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947],
        [-0.8285,  0.3947]])


RuntimeError: a Tensor with 32 elements cannot be converted to Scalar

In [96]:
# training loop
for epoch in range(epochs):
    base_torch_model.train()
    loss_run = 0.0
    for images, labels in train_load:
        images, labels = images.to(device), labels.squeeze().to(device)
        optimizer.zero_grad()
        out = base_torch_model(images)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        loss_run += loss.item()
    loss_res = loss_run / len(train_load)

    base_torch_model.eval()
    true, all = 0, 0
    with torch.no_grad():
        for images, labels in val_load:
            images, labels = images.to(device), labels.squeeze().to(device)
            out = base_torch_model(images)
            _, predicted = torch.max(out.data, 1)
            all += labels.size(0)
            true += (predicted == labels.squeeze()).sum().item()
    acc = true / all
    print('acc', acc)

RuntimeError: Given groups=1, weight of size [3, 3, 3, 3], expected input[32, 100, 100, 1] to have 3 channels, but got 100 channels instead

In [ ]:
# testing loop
base_torch_model.eval()
true, all = 0, 0
with torch.no_grad():
    for images, labels in test_load:
        images, labels = images.to(device), labels.squeeze().to(device)
        out = base_torch_model(images)
        _, predicted = torch.max(out.data, 1)
        all += labels.size(0)
        true += (predicted == labels.squeeze()).sum().item()
acc = true / all
print('acc', acc)